In [1]:
import pandas
import matplotlib
path = 'analysisOutput/results'

In [2]:
pandas.read_csv?

In [3]:
def get_cols_list(path: str):
    cols = pandas.read_csv(path, index_col=None, delimiter=" ")
    return list(cols.columns.values)

dp_cols = get_cols_list(f'{path}/DP/column_description_DP.dat')
greedy_cols = get_cols_list(f'{path}/Greedy/column_description_Greedy.dat')
greedy_one_cols = get_cols_list(f'{path}/GreedyOne/column_description_GreedyOne.dat')
fptas_cols = get_cols_list(f'{path}/FPTAS/column_description_FPTAS.dat')

dp_cols

['id', 'item_count', 'maximum_sum', 'time', '|', 'items_in_bag']

In [4]:
dp_test = pandas.read_csv('analysisOutput/results/DP/NK4_inst_DP.dat', index_col=None, delimiter=" ", header=None)
dp_test

,0,1,2,3,4,5,6,7,8
0,1,4,1129,20.716860,|,0,1,0,0
1,2,4,2966,24.251711,|,0,1,1,0
2,3,4,0,63.193333,|,0,0,0,0
3,4,4,70,34.024672,|,1,0,0,0
4,5,4,2758,31.722900,|,0,1,0,1
...,...,...,...,...,...,...,...,...,...
495,496,4,3106,38.453033,|,0,1,1,1
496,497,4,4426,48.939436,|,1,1,1,0
497,498,4,2145,42.883349,|,0,0,0,1
498,499,4,2036,51.440695,|,0,1,0,0


In [5]:
dp_cols
#dp_test.columns = dp_cols
#dp_test["5"] = dp_test["6"] + dp_test["7"] + dp_test["8"]
dp_test[5] = dp_test[5].map(str) + dp_test[6].map(str) + dp_test[7].map(str) + dp_test[8].map(str)
dp_test.drop(["|"], axis=1)
dp_test

KeyError: "['|'] not found in axis"